In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## For data
import pandas as pd
import numpy as np
import random

## For plotting
import matplotlib.pyplot as plt

## For machine learning
from sklearn import preprocessing, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

%matplotlib inline

## Chinese display
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

## No warnings
import warnings
warnings.filterwarnings('ignore') 

In [57]:
def eval(X_test, y_test, predicted, predicted_prob, show=False):

    # Accuray e AUC
    accuracy = metrics.accuracy_score(y_test, predicted)
    auc = metrics.roc_auc_score(y_test, predicted_prob)
    if show:
        print("Accuracy (overall correct predictions):",  round(accuracy, 4))
        print("Auc:", round(auc, 4))

    # Precision & Recall
    recall = metrics.recall_score(y_test, predicted)
    precision = metrics.precision_score(y_test, predicted)
    if show:
        print("Recall (all 1s predicted right):", round(recall, 4))
        print("Precision (confidence when predicting a 1):", round(precision, 4))
        print("Detail:")
        print(metrics.classification_report(y_test, predicted,
            target_names=[str(i) for i in np.unique(y_test)]))
    
    # f-1
    precision_, recall_, f_1, support = precision_recall_fscore_support(
        y_true=y_test, y_pred=predicted,beta=1)
    if show:
        print(f'F1 value: {f_1[1]}')

    # f-beta
    precision_, recall_, f_beta, support = precision_recall_fscore_support(
        y_true=y_test, y_pred=predicted,beta=1.5)
    if show:
        print(f'F-beta value: {f_beta[1]}')

    # # confusion matrix
    # classes = np.unique(y_test)
    # fig, ax = plt.subplots(figsize=(7, 5))
    # cm = metrics.confusion_matrix(y_test, predicted, labels=classes)
    # sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False)
    # ax.set(xlabel="Pred", ylabel="True", title="Confusion matrix")
    # ax.set_yticklabels(labels=classes, rotation=0)
    # plt.tight_layout()
    # plt.show()

    return [recall, precision, f_1[1], f_beta[1]]


# f_beta
def calculate_fscore(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        Y,
                                                        test_size=0.3,
                                                        stratify=Y,
                                                        random_state=42)                                          
    model = linear_model.LogisticRegression(class_weight='balanced', solver='lbfgs')
    model.fit(X_train, y_train)

    predicted_prob = model.predict_proba(X_test)[:, 1]
    predicted = model.predict(X_test)
    test_metrics = eval(X_test, y_test, predicted, predicted_prob, True)

    return test_metrics

## Feature selection
### 1. No feature selection
#### 1.1 One-hot encoding

In [58]:
df_file = "data/Diarrhea_fillna.tsv"
df1 = pd.read_csv(df_file, sep="\t", index_col=0, encoding="utf-8")

y = "细菌结果"
df1[y] = df1[y].map(lambda x: 0 if x == "阴性" else 1)

df1 = pd.get_dummies(df1, drop_first=True)
df1.shape

X_original = df1.drop(y, axis=1)
y_original = df1[y]

# [recall, precision, f_1[1], f_beta[1]]
test_metrics = calculate_fscore(X_original, y_original)
# print("No feature selection: f_beta of one-hot encoding: ", f_beta_original)
# print(test_metrics)

(11600, 383)

Accuracy (overall correct predictions): 0.7098
Auc: 0.7471
Recall (all 1s predicted right): 0.616
Precision (confidence when predicting a 1): 0.3728
Detail:
              precision    recall  f1-score   support

           0       0.88      0.73      0.80      2769
           1       0.37      0.62      0.46       711

    accuracy                           0.71      3480
   macro avg       0.63      0.67      0.63      3480
weighted avg       0.78      0.71      0.73      3480

F1 value: 0.46447507953340406
F-beta value: 0.5130191909181008


#### 1.2 Feature embedding

In [59]:
df_file = "data/Diarrhea_embed_100.tsv"
df = pd.read_csv(df_file, sep="\t", index_col=0, encoding="utf-8")
df.shape

y = "细菌结果"
X_embed = df.drop(y, axis=1)
y_embed = df[y]
test_metrics = calculate_fscore(X_embed, y_embed)

# print("No feature selection: f_beta of feature embedding: ", f_beta_embed)

(11600, 101)

Accuracy (overall correct predictions): 0.704
Auc: 0.7397
Recall (all 1s predicted right): 0.6118
Precision (confidence when predicting a 1): 0.3659
Detail:
              precision    recall  f1-score   support

         0.0       0.88      0.73      0.80      2769
         1.0       0.37      0.61      0.46       711

    accuracy                           0.70      3480
   macro avg       0.62      0.67      0.63      3480
weighted avg       0.77      0.70      0.73      3480

F1 value: 0.45789473684210524
F-beta value: 0.5069475571492604


#### 1.3 One-hot encoding + feature embedding

In [60]:
df_file = "data/Diarrhea_fillna.tsv"
df = pd.read_csv(df_file, sep="\t", index_col=0, encoding="utf-8")
y = "细菌结果"
df[y] = df[y].map(lambda x: 0 if x == "阴性" else 1)
df_onehot = pd.get_dummies(df, drop_first=True)
df_onehot = df_onehot.reset_index(drop=True)
df_onehot.shape

df_file = "data/Diarrhea_embed_100.tsv"
df_embed = pd.read_csv(df_file, sep="\t", index_col=0, encoding="utf-8")
y = "细菌结果"
df_embed = df_embed.drop(y, axis=1)
df_embed.shape

df = pd.concat([df_onehot, df_embed], axis=1)
df.shape

df_combined = "data/Diarrhea_onehot_embed_482.tsv"
df.to_csv(df_combined, sep="\t", encoding="utf-8")

(11600, 383)

(11600, 100)

(11600, 483)

In [61]:
df_file = "data/Diarrhea_onehot_embed_482.tsv"
df = pd.read_csv(df_file, sep="\t", index_col=0, encoding="utf-8")
df.shape

y = "细菌结果"
X_embed = df.drop(y, axis=1)
y_embed = df[y]
test_metrics = calculate_fscore(X_embed, y_embed)

# print("No feature selection: f_beta of one-hot encoding and feature embedding: ", df_onehot_embed)

(11600, 483)

Accuracy (overall correct predictions): 0.7046
Auc: 0.74
Recall (all 1s predicted right): 0.616
Precision (confidence when predicting a 1): 0.3671
Detail:
              precision    recall  f1-score   support

           0       0.88      0.73      0.80      2769
           1       0.37      0.62      0.46       711

    accuracy                           0.70      3480
   macro avg       0.62      0.67      0.63      3480
weighted avg       0.78      0.70      0.73      3480

F1 value: 0.46008403361344535
F-beta value: 0.5097126488228448


### 2. Statistical-based methods
#### 2.1 One-hot encoding 

In [62]:
df_file = "data/Diarrhea_fillna.tsv"
df2 = pd.read_csv(df_file, sep="\t", index_col=0, encoding="utf-8")

y = "细菌结果"
df2[y] = df2[y].map(lambda x: 0 if x == "阴性" else 1)

# relevant features from 'main.ipynb'
features_chi2_anova = [
    'age', '体温', '腹泻量', '腹泻频次', '腹泻天数', '细菌结果', '区县', '性别', '户籍', '职业', '首发症状',
    '发热', '腹胀', '恶心', '腹痛', '腹痛性质', '腹痛部位', '呕吐在腹泻___发生', '腹泻', '腹泻性质',
    '近6个月有无肠道疾病既往史', '进餐地点', '是否家中饲养或接触过宠物', '就诊前是否服用过抗生素',
    '诊断', '诊断类型', '临床处理', '本次就诊是否给予抗生素', '抗生素名称.1', '是否采集', '采样类型'
]

df2 = df2[features_chi2_anova]
df2 = pd.get_dummies(df2, drop_first=True)
df2.shape

X_chi2_anova = df2.drop(y, axis=1)
y_chi2_anova = df2[y]
test_metrics = calculate_fscore(X_chi2_anova, y_chi2_anova)

# print("Statistical-based method: f_beta of one-hot encoding: ", f_beta_chi2_anova)

(11600, 358)

Accuracy (overall correct predictions): 0.7069
Auc: 0.7469
Recall (all 1s predicted right): 0.6203
Precision (confidence when predicting a 1): 0.3703
Detail:
              precision    recall  f1-score   support

           0       0.88      0.73      0.80      2769
           1       0.37      0.62      0.46       711

    accuracy                           0.71      3480
   macro avg       0.63      0.67      0.63      3480
weighted avg       0.78      0.71      0.73      3480

F1 value: 0.46372239747634064
F-beta value: 0.513571620532115


### 3. Reinforcement Learning-based methods

In [ ]:
df_file = "data/Diarrhea_fillna.tsv"
df = pd.read_csv(df_file, sep="\t", index_col=0, encoding="utf-8")
df.shape

(11600, 46)

In [ ]:
y = "细菌结果"
df[y] = df[y].map(lambda x: 0 if x == "阴性" else 1)
df[y].value_counts()

0    9231
1    2369
Name: 细菌结果, dtype: int64

In [ ]:
# one-hot encoding
df = pd.get_dummies(df, drop_first=True)
df.shape
# 382 features (excluding the label column)

(11600, 383)

In [ ]:
y = '细菌结果'
names = df.columns
scaler = preprocessing.MinMaxScaler().fit(df)
df = scaler.transform(df)
df = pd.DataFrame(df, columns=names)

df_normal_file = "data/Diarrhea_onehot_382.tsv"
df.to_csv(df_normal_file, sep="\t", encoding="utf-8")

#### 3.1 One-hot encoding

To repeat the RL-based experiment, you can run it from here.

In [ ]:
df_file = "data/Diarrhea_onehot_382.tsv"
df = pd.read_csv(df_file, sep="\t", index_col=0, encoding="utf-8")
df.shape

(11600, 383)

#### 3.2 One-hot encoding + feature embedding

In [ ]:
# df_file = "data/Diarrhea_onehot_embed_482.tsv"
# df = pd.read_csv(df_file, sep="\t", index_col=0, encoding="utf-8")
# df.shape

Choose any one of the above three feature encoding methods.

In [ ]:
# Stratified sampling
# train set : test set = 7 : 3 
y = '细菌结果'
df_train, df_test = train_test_split(df, test_size=0.3,
                                     stratify=df[y], random_state=42)


# print info
print("X_train shape:", df_train.drop(y, axis=1).shape,
      "| X_test shape:", df_test.drop(y, axis=1).shape,)
print("y_train mean:", round(
    np.mean(df_train[y]), 2), "| y_test mean:", round(np.mean(df_test[y]), 2))

print('-'*50)

print("Train set：")
print(df_train[y].value_counts() / len(df_train[y]))
print("Test set：")
print(df_test[y].value_counts() / len(df_test[y]))

X_train shape: (8120, 382) | X_test shape: (3480, 382)
y_train mean: 0.2 | y_test mean: 0.2
--------------------------------------------------
Train set：
0.0    0.795813
1.0    0.204187
Name: 细菌结果, dtype: float64
Test set：
0.0    0.79569
1.0    0.20431
Name: 细菌结果, dtype: float64


In [ ]:
y = '细菌结果'
features_raw = df.drop(y, axis=1).columns.to_list()

X_train = df_train[features_raw]
y_train = df_train[y]

X_test = df_test[features_raw]
y_test = df_test[y]

In [ ]:
# f_beta
def accuracy(input, show=False):
    x_train = X_train.iloc[:, input]
    x_test = X_test.iloc[:, input]
    clf = linear_model.LogisticRegression(class_weight='balanced')
    clf.fit(x_train, y_train.values.ravel())


    predicted_prob = clf.predict_proba(x_test)[:, 1]
    predicted_test = clf.predict(x_test)
    test_metrics = eval(x_test, y_test, predicted_test, predicted_prob, show)
    # beta = 1.5
    precision, recall, f_beta_test, support = precision_recall_fscore_support(
        y_true=y_test, y_pred=predicted_test,beta=1.5)

    return f_beta_test[1]

In [ ]:
## Initialize Q-table with -1
# Q_values = [[-1, -1]] * len(features_raw)
Q_values = [[-1, -1] for i in range(len(features_raw))]
Q_values[:3]

## How to add a priori weights to the features? 
# (run "Statistical-based methods: One-hot encoding" first and then run the following code)
# 1. get the column names of the columns of df2, or X_chi2_anova, so that there are no y label
# 2. find the subscripts i corresponding to these column names in df
# 3. set the second column of row i in Q_values to 0 > 1

# add a priori weights to the features
# for ix, column in enumerate(X_train.columns):
#     if column in X_chi2_anova.columns:
#         Q_values[ix][1] = 0

# Q_values[:3]

[[-1, -1], [-1, -1], [-1, -1]]

In [ ]:
def get_reward(features):
    if len(features) == 0:
        return 0
    temp = accuracy(features)
    acc = temp * 100
    tot_f = len(features)
    R = acc
    if tot_f > K:
        R = acc * K / tot_f
    return R

In [ ]:
# epsilon-greedy policy
epsilon = 0.5
alpha = 0.2

# attenuation coefficient
epsilon_decay_rate = 0.995
alpha_decay_rate = 0.995

# Maximum number of features allowed
K = 400

In [ ]:
all_rewards = []
num_episodes = 100

# Assign an agent to each feature
num_agents = len(features_raw)

# Initialize the reward matrix with 0
reward_store = {}
for i in range(num_agents + 1):
    reward_store[i] = [0, []] 

In [ ]:
# Initialize the action space with 0
actions = [0] * num_agents
from tqdm import tqdm
for episode in tqdm(range(num_episodes)):
    for agent in range(num_agents):
        rand_number = random.uniform(0, 1)
        if rand_number > epsilon:
            # actions[agent]  = Q_values[agent].index(max(Q_values[agent]))
            actions[agent] = np.argmax(Q_values[agent])
        else:
            actions[agent] = random.choice([0, 1])
    features = []
    for i, act in enumerate(actions):
        if act == 1:
            features.append(i)
    # print(features)
    R = get_reward(features)
    if R > reward_store[len(features)][0]:
        reward_store[len(features)] = [R, features]
    # print(R)
    all_rewards.append(R)

    for agent in range(num_agents):
        actions[agent] = 1 - actions[agent] 
        features = []
        for i, act in enumerate(actions):
            if act == 1:
                features.append(i)
        # CLEAN Reward：C_i = G(a-a_i+c_i) - G(a)
        C_agent = get_reward(features) - R
        Q_values[agent][actions[agent]] = Q_values[agent][actions[agent]] + alpha * (C_agent - Q_values[agent][actions[agent]])
    alpha = alpha * alpha_decay_rate
    epsilon = epsilon * epsilon_decay_rate

100%|██████████| 100/100 [2:20:15<00:00, 84.16s/it] 


In [ ]:
# Feature dimension: 202
max(reward_store,key=reward_store.get)

207

In [ ]:
# The best performer in the test set
print("RL-based method in test set:", reward_store[max(reward_store,key=reward_store.get)])

RL-based method in test set: [52.92531679698031, [2, 3, 4, 6, 9, 11, 12, 13, 14, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 33, 34, 35, 36, 38, 39, 41, 44, 45, 46, 47, 48, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 63, 65, 69, 70, 71, 72, 74, 78, 80, 82, 84, 87, 89, 91, 92, 93, 99, 102, 103, 105, 106, 107, 110, 114, 115, 116, 119, 120, 121, 122, 124, 125, 128, 131, 132, 135, 136, 137, 139, 142, 144, 146, 147, 149, 151, 152, 153, 154, 155, 158, 163, 164, 165, 169, 170, 172, 174, 175, 179, 180, 184, 186, 187, 188, 190, 192, 194, 195, 198, 201, 204, 206, 209, 210, 211, 213, 214, 216, 217, 218, 219, 221, 222, 223, 226, 228, 230, 231, 234, 235, 239, 240, 241, 242, 244, 250, 251, 252, 253, 255, 256, 259, 262, 263, 264, 266, 267, 269, 271, 272, 274, 278, 283, 284, 286, 287, 288, 290, 292, 294, 295, 298, 299, 300, 304, 307, 314, 315, 317, 319, 324, 325, 326, 327, 328, 331, 332, 333, 335, 337, 338, 341, 342, 344, 346, 347, 348, 349, 351, 352, 355, 357, 358, 363, 364, 366, 367, 370, 371, 373, 375,

In [ ]:
# Extract the most effective subset according to the Q table and test it with the test set!

def get_final_reward(features):
    if len(features) == 0:
        return 0
    f_beta = accuracy(features, True)
    return f_beta

# use Q-table
for agent in range(num_agents):
        actions[agent] = np.argmax(Q_values[agent])
   
features = []
for i, act in enumerate(actions):
    if act == 1:
        features.append(i)

# use experience
# features = reward_store[max(reward_store,key=reward_store.get)][1]
# R = get_reward(features)

R = get_final_reward(features)

print("RL-based method in test set: ", R)
print("final size of feature subset", len(features))


Accuracy (overall correct predictions): 0.7164
Auc: 0.751
Recall (all 1s predicted right): 0.6371
Precision (confidence when predicting a 1): 0.3832
Detail:
              precision    recall  f1-score   support

         0.0       0.89      0.74      0.81      2769
         1.0       0.38      0.64      0.48       711

    accuracy                           0.72      3480
   macro avg       0.64      0.69      0.64      3480
weighted avg       0.78      0.72      0.74      3480

F1 value: 0.4786053882725832
F-beta value: 0.5292531679698032
RL-based method in test set:  0.5292531679698032
final size of feature subset 207
